In [1]:
import cv2
import mediapipe as mp
import numpy as np

# Mediapipe drawing
mp_drawing = mp.solutions.drawing_utils
draw_spec = mp_drawing.DrawingSpec(thickness=1, circle_radius=1)
# Mediapipe API --> holistic
mp_holistic = mp.solutions.holistic

# For webcam solution
holistic = mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5)

In [2]:
# Open camera via openCV
cap = cv2.VideoCapture(0)

while cap.isOpened():
  success, image = cap.read()

  # Flip image horizontally for selfie-view display & convert color space form BGR to RGB
  image = cv2.cvtColor(cv2.flip(image, 1), cv2.COLOR_BGR2RGB)
  # To improve perormance
  image.flags.writeable = False

  # Get the result for mediapipe-holistic
  results = holistic.process(image)

  image.flags.writeable = True

  # convert color space form RGB to BGR
  image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

  img_h, img_w, img_c = image.shape
  rotation_3D = []
  rotation_2D = []

  # Extract landmarks and calculate face rotation angles
  if results.face_landmarks:
    for idx, lm in enumerate(results.face_landmarks.landmark):
      if idx == 33 or idx == 263 or idx == 1 or idx == 61 or idx == 291 or idx == 199:
        if idx == 1:
          nose_2d = (lm.x * img_w, lm.y * img_h)
          nose_3d = (lm.x * img_w, lm.y * img_h, lm.z * 3000)

        x, y = int(lm.x * img_w), int(lm.y * img_h)

        # Get the 3D Coordinates
        rotation_3D.append([x, y, lm.z])
        # Get the 2D Coordinates
        rotation_2D.append([x, y])

    # Convert it to the NumPy array 
    rotation_3D = np.array(rotation_3D, dtype=np.float64)
    rotation_2D = np.array(rotation_2D, dtype=np.float64)

    # The camera matrix
    focal_length = 1 * img_w

    cam_matrix = np.array(
      [[focal_length, 0, img_h / 2],
      [0, focal_length, img_w / 2],
      [0, 0, 1]])

    # The Distance Matrix
    dist_matrix = np.zeros((4, 1), dtype=np.float64)

    # Solve PnP
    success, rot_vec, trans_vec = cv2.solvePnP(rotation_3D, rotation_2D, cam_matrix, dist_matrix)

    # Get rotation
    rmat, jac = cv2.Rodrigues(rot_vec)
    
    # Get angles
    angles, mtxR, mtxQ, Qx, Qy, Qz = cv2.RQDecomp3x3(rmat)

    # Get x, y, z angles
    x = angles[0] * 360
    y = angles[1] * 360
    z = angles[2] * 360

    nose_3d_projection, jacobian = cv2.projectPoints(nose_3d, rot_vec, trans_vec, cam_matrix, dist_matrix)

    p1 = (int(nose_2d[0]), int(nose_2d[1]))
    p2 = (int(nose_2d[0] + y * 10), int(nose_2d[1] - x * 10 ))

    cv2.line(image, p1, p2, (255, 0, 0), 3)

    cv2.putText(image, "x: " + str(np.round(x, 2)), (1000, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 3)
    cv2.putText(image, "y: " + str(np.round(y, 2)), (1000, 100), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 3)
    cv2.putText(image, "z: " + str(np.round(z, 2)), (1000, 150), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 3)

    mp_drawing.draw_landmarks(
          image = image,
          landmark_list = results.face_landmarks,
          connections = mp_holistic.FACEMESH_TESSELATION,
          landmark_drawing_spec = draw_spec,
          connection_drawing_spec = draw_spec)

    cv2.imshow('Head Pose Estimation', image)

  if cv2.waitKey(5) & 0xFF == 27:
    break

cap.release()
cv2.destroyAllWindows()

In [ ]:
def vec_length(v: np.array):
    return np.sqrt(sum(i**2 for i in v))

def normalize(v):
    norm = np.linalg.norm(v)
    if norm == 0: 
       return v
       print(v, norm)
    return v / norm

def look_at(eye: np.array, target: np.array):
    axis_z = normalize((eye - target))
    if vec_length(axis_z) == 0:
        axis_z = np.array((0, -1, 0))
        
    axis_x = np.cross(np.array((0, 0, 1)), axis_z)
    if vec_length(axis_x) == 0:
        axis_x = np.array((1, 0, 0))
    
    axis_y = np.cross(axis_z, axis_x)
    rot_matrix = np.matrix([axis_x, axis_y, axis_z]).transpose()
    return rot_matrix

normal = results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_EYE]
angle30 = results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_EYE]

orient = np.rad2deg(look_at(np.array([normal.x, normal.y, normal.z]), 
    np.array([angle30.x, angle30.y, angle30.z])))

print(orient)

In [ ]:
# Euler Angles
import math

thedaX = math.atan2(orient.item(2,1), orient.item(2,2))
thedaY = math.atan2(-orient.item(2,0), (orient.item(2,1)**2 + orient.item(2,2)**2)**0.5)
thedaZ = math.atan2(orient.item(1,0), orient.item(0,0))

print("thedaX: ", thedaX)
print("thedaY: ", thedaY)
print("thedaZ: ", thedaZ)

In [13]:
baselineLandmark = []

baselineLandmark.append([1,2,3,4])

baselineLandmark.append([2,2,3,4])


In [14]:
print(baselineLandmark)

[[1, 2, 3, 4], [2, 2, 3, 4]]


In [16]:
print(baselineLandmark[0][1])

2


In [1]:
count = 0
if count%15 ==0:
    print("yes")
else:
    print("no")

yes


In [2]:
import pandas as pd

dataTemp1 = pd.read_csv("/Users/ray/Thesis/ShoulderAll_0721.csv")
dataTemp2 = pd.read_csv("/Users/ray/Thesis/ShoulderAll_0722.csv")

dataAll = pd.merge(dataTemp1, dataTemp2, how='outer')

trainData = dataAll.reset_index().sort_values(['ID'])

trainData.to_csv('ShoulderAll_0721_add.csv', index=False)

KeyboardInterrupt: 